In [1]:
!pip install --upgrade tensorflow-gpu keras tqdm
import tensorflow as tf
tf.__version__, tf.test.gpu_device_name()

     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 378kB 39.4MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 3.8MB 46.1MB/s 
     |████████████████████████████████| 450kB 57.7MB/s 
     |████████████████████████████████| 81kB 13.5MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.1 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorb

('2.1.0', '/device:GPU:0')

In [2]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/My Drive/External-Tech/Codes/bert')
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/My Drive/External-Tech/Codes/bert']

In [0]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

from tqdm import tqdm
from tokenization import FullTokenizer

In [0]:
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/Kaggle/nlp-getting-started/input'

BERT_PATH = '/content/drive/My Drive/External-Tech/Models/bert-base-uncased-tf'
TOKENIZER = FullTokenizer(os.path.join(BERT_PATH, 'assets', 'vocab.txt'), True)

In [6]:
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train_extra_feat.csv'))
df_train.head()

,id,keyword,location,text,target,word_count,unique_word_count,stop_word_count,non_stop_word_count,url_count,hashtag_count,mention_count,char_count,punctuation_count,sentence_count,mean_word_length,mean_sentence_length,number_count,clean_text,typo_count,slang_count
0,1,no%20keyword,no%20location,Our Deeds are the Reason of this #earthquake M...,1,13,13,6,7,0,1,0,69,1,1,4.384615,69.0,0,Our Deeds are the Reason of this earthquake Ma...,0,0
1,4,no%20keyword,no%20location,Forest fire near La Ronge Sask. Canada,1,7,7,0,7,0,0,0,38,1,2,4.571429,18.5,0,Forest fire near La Ronge Sask. Canada,2,0
2,5,no%20keyword,no%20location,All residents asked to 'shelter in place' are ...,1,22,20,11,11,0,0,0,133,3,2,5.090909,66.0,0,All residents asked to ishelter in place' are ...,3,0
3,6,no%20keyword,no%20location,"13,000 people receive #wildfires evacuation or...",1,8,8,1,7,0,1,0,65,2,1,7.125000,64.0,0,"13,000 people receive wildfires evacuation ord...",2,0
4,7,no%20keyword,no%20location,Just got sent this photo from Ruby #Alaska as ...,1,16,15,7,9,0,2,0,88,2,1,4.500000,87.0,0,Just got sent this photo from Ruby Alaska as s...,1,0


In [7]:
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test_extra_feat.csv'))
df_test.head()

,id,keyword,location,text,word_count,unique_word_count,stop_word_count,non_stop_word_count,url_count,hashtag_count,mention_count,char_count,punctuation_count,sentence_count,mean_word_length,mean_sentence_length,number_count,clean_text,typo_count,slang_count,emoji_count
0,0,no%20keyword,no%20location,Just happened a terrible car crash,6,6,2,4,0,0,0,34,0,1,4.833333,34.0,0,Just happened a terrible car crash,0,0,0
1,2,no%20keyword,no%20location,"Heard about #earthquake is different cities, s...",9,9,2,7,0,1,0,64,3,1,6.222222,64.0,0,"Heard about earthquake is different cities, st...",2,0,0
2,3,no%20keyword,no%20location,"there is a forest fire at spot pond, geese are...",19,19,10,9,0,0,0,96,2,1,4.105263,96.0,0,"there is a forest fire at spot pond, geese are...",3,0,0
3,9,no%20keyword,no%20location,Apocalypse lighting. #Spokane #wildfires,4,4,0,4,0,2,0,40,3,2,9.250000,19.5,0,Apocalypse lighting. Spokane wildfires,3,0,0
4,11,no%20keyword,no%20location,Typhoon Soudelor kills 28 in China and Taiwan,8,8,2,6,0,0,0,45,0,1,4.750000,45.0,1,Typhoon Soudelor kills 28 in China and Taiwan,1,0,0


In [8]:
df_train.shape, df_test.shape

((7613, 21), (3263, 21))

In [9]:
df_train['word_count'].max()

31

In [10]:
df_test['word_count'].max()

31

In [0]:
MAX_SEQUENCE_LENGTH = 50

In [0]:
def tokenize(input):
    return TOKENIZER.tokenize(input)

In [0]:
def get_ids(tokenized_text):
    token_ids = TOKENIZER.convert_tokens_to_ids(tokenized_text)
    input_ids = token_ids + [0] * (MAX_SEQUENCE_LENGTH - len(token_ids))
    return input_ids

In [0]:
def get_masks(tokenized_text):
    return [1] * len(tokenized_text) + [0] * (MAX_SEQUENCE_LENGTH - len(tokenized_text))

In [0]:
def get_segments(tokenized_text):
    return [0] * MAX_SEQUENCE_LENGTH

In [0]:
def convert_to_bert_inputs(tokenized_text):
    if len(tokenized_text) > MAX_SEQUENCE_LENGTH - 2:
        print('Removing extra in ...')
        print(len(tokenized_text), tokenized_text)
        print()
        tokenized_text = tokenized_text[:MAX_SEQUENCE_LENGTH - 2]
    tokens = ['[CLS]'] + tokenized_text + ['[SEP]']
    input_ids = get_ids(tokens)
    input_masks = get_masks(tokens)
    input_segments = get_segments(tokens)
    return input_ids, input_masks, input_segments

In [0]:
def compute_input_arrays(df, col):
    input_ids, input_masks, input_segments = [], [], []
    with tqdm(total=len(df[col].values)) as pbar:
        for row in df[col].values:
            tokenized_text = TOKENIZER.tokenize(row)
            ids, masks, segments = convert_to_bert_inputs(tokenized_text)
            input_ids.append(ids)
            input_masks.append(masks)
            input_segments.append(segments)
            pbar.update(1)
            
    return [np.asarray(input_ids, dtype=np.int32),
            np.asarray(input_masks, dtype=np.int32),
            np.asarray(input_segments, dtype=np.int32)]

In [46]:
inputs_train = compute_input_arrays(df_train, 'clean_text')

 16%|█▋        | 1249/7613 [00:00<00:01, 4035.00it/s]

Removing extra in ...
54 ['ca', '##go', '##v', 'if', '90', '##bl', '##ks', 'and', 'amp', '8', '##w', '##ht', '##s', 'col', '##lu', '##ded', '2', 'take', 'w', '##ht', 'f', 'usa', '##go', '##v', 'au', '##th', 'hostage', 'and', 'amp', '2', 'make', 'her', 'look', 'b', '##lk', 'w', 'bio', '##ter', '##ror', '##ism', 'and', 'amp', 'use', 'her', 'l', '##gl', 'org', 'id', '##is', 'id', 'still', 'hers', '?', 'vp']

Removing extra in ...
49 ['ap', '##hia', '##bet', '##a1', '##90', '##7', 'w', 'u', '##gli', '##ness', 'due', '2', 'your', "'", 'ugly', "'", 'ames', '##oc', '##ial', '##act', '##ion', 'fra', '##t', 'is', 'bio', '##ter', '##ror', '##ism', '##i', 'am', 'she', 'who', 'is', 'fbi', 'id', 'you', '$', 'to', '##le', '##wan', '##t', "'", 'g', 'another', 'in', 'my', 'home', '.', 'abc']

Removing extra in ...
52 ['howard', '##u', 'if', '90', '##bl', '##ks', 'and', 'amp', '8', '##w', '##ht', '##s', 'col', '##lu', '##ded', '2', 'take', 'w', '##ht', 'f', 'usa', '##go', '##v', 'au', '##th', 'hostage'

 33%|███▎      | 2476/7613 [00:00<00:01, 4064.37it/s]

Removing extra in ...
62 ['info', 's', '.', 'w', '##nd', '03', '##0', '6', '.', 'cl', '##d', 'sc', '##t', '##01', '##4', 'bk', '##n', '##0', '##32', '.', 'ex', '##p', 'ins', '##t', 'ap', '##ch', '.', 'r', '##wy', '05', '.', 'cu', '##rf', '##ew', 'in', 'op', '##er', 'until', '203', '##0', 'z', '.', 'taxi', '##ways', 'fox', '##tro', '##t', '5', 'and', 'amp', 'fox', '##tro', '##t', '6', 'na', '##vb', '##l', '.', 't', '##mp', '10', '.']

Removing extra in ...
58 ['info', 'r', '.', 'cu', '##rf', '##ew', 'in', 'op', '##er', 'until', '203', '##0', 'z', '.', 'taxi', '##ways', 'fox', '##tro', '##t', '5', 'and', 'amp', 'fox', '##tro', '##t', '6', 'na', '##vb', '##l', '.', 'w', '##nd', '06', '##0', '5', '.', 'ex', '##p', 'ins', '##t', 'ap', '##ch', '.', 'r', '##wy', '05', '.', 'damp', '.', 't', '##mp', '10', '.', 'q', '##nh', '102', '##8', '.']

Removing extra in ...
62 ['info', 'u', '.', 'cl', '##d', 'sc', '##t', '##01', '##2', 'bk', '##n', '##0', '##25', '.', 'ex', '##p', 'ins', '##t', 'ap', '#

100%|██████████| 7613/7613 [00:01<00:00, 4161.33it/s]


In [47]:
inputs_test = compute_input_arrays(df_test, 'clean_text')

 25%|██▍       | 802/3263 [00:00<00:00, 3954.29it/s]

Removing extra in ...
53 ['usc', '##our', '##t', 'if', '90', '##bl', '##ks', 'and', 'amp', '8', '##w', '##ht', '##s', 'col', '##lu', '##ded', '2', 'take', 'w', '##ht', 'f', 'usa', '##go', '##v', 'au', '##th', 'hostage', 'and', 'amp', '2', 'make', 'her', 'look', 'b', '##lk', 'w', 'bio', '##ter', '##ror', '##ism', 'and', 'amp', 'use', 'her', 'l', '##gl', 'org', 'id', '##is', 'id', 'still', 'hers', '?']

Removing extra in ...
52 ['harvard', '##u', 'if', '90', '##bl', '##ks', 'and', 'amp', '8', '##w', '##ht', '##s', 'col', '##lu', '##ded', '2', 'take', 'w', '##ht', 'f', 'usa', '##go', '##v', 'au', '##th', 'hostage', 'and', 'amp', '2', 'make', 'her', 'look', 'b', '##lk', 'w', 'bio', '##ter', '##ror', '##ism', 'and', 'amp', 'use', 'her', 'l', '##gl', 'org', 'id', '##is', 'id', 'still', 'hers', '?']

Removing extra in ...
53 ['ga', '##court', '##s', 'if', '90', '##bl', '##ks', 'and', 'amp', '8', '##w', '##ht', '##s', 'col', '##lu', '##ded', '2', 'take', 'w', '##ht', 'f', 'usa', '##go', '##v',

100%|██████████| 3263/3263 [00:00<00:00, 3904.43it/s]


In [61]:
inputs_train[0].shape

(7613, 50)

In [0]:
input_word_ids = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32,
                                       name='input_word_ids')
input_masks = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, 
                                    name='input_masks')
input_segments = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32,
                                       name='input_segments')

bert_layer = hub.KerasLayer(BERT_PATH, trainable=True)
_, sequence_output = bert_layer([input_word_ids, input_masks, input_segments])
output = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)

model = tf.keras.models.Model(
    inputs=[input_word_ids, input_masks, input_segments], 
    outputs=[output])

In [0]:
outputs_train = model.predict(inputs_train)

In [0]:
outputs_test = model.predict(inputs_test)

In [0]:
np.save(os.path.join(DATA_PATH, 'bert-embeddings-train.npy'), outputs_train)

In [0]:
np.save(os.path.join(DATA_PATH, 'bert-embeddings-test.npy'), outputs_test)